# **Power System Simulation Final Presentation FUSE**

## Introduction
Welcome to the final presentation on Power System Simulation. In this presentation, we will cover the following packages:

* `graph_processing.py`
* `power_grid_modelling.py`
* `grid_analytic.py`

We have tested all the packages using `pytest` to ensure there are no errors and that the output is as expected. For performance checking, we will also run a timer to check how fast the package would run

To be able to use the package, users must install the following dependencies (libraries):
* `numpy`
* `pandas`
* `networkx`
* `power-grid-model`
* `math`
* `random`

In [3]:
import time

class Timer:
    def __init__(self, name: str):
        self.name = name
        self.start = None

    def __enter__(self):
        self.start = time.perf_counter()

    def __exit__(self, *args):
        print(
            f'Execution time for {self.name} is {(time.perf_counter() - self.start):0.6f} s'
        )

# **1.Graph-Processing**

This section covers the functionalities and usage of the `graph_processing.py` package. The `graph_processing.py` package builds an undirected graph and implements two functionalties namely, `find_downstream_vertices()` and `find_alternative_edges()`.

## **1.1 Input data**

To create a graph the following parameters need to be defined:
* `edge_ids`
* `edge_vertex_id_pairs`
* `edge_enabled`
* `source_vertex_id`

From that, the graph processing package can be called as follow.


In [4]:
from power_system_simulation.graph_processing import GraphProcessor

For example, assume you have a network that is built as described below:

      ------- edge_1---- vertex_0 ------ edge_5 -----
      |                    |                       |
      |                  edge_3                    |
      |                    |                       |
    vertex_2 -- edge_7 -- vertex_4  -- edge_8 --vertex_6
      |
    edge_9
      |
    vertex_10  

If only the enabled edges are taken into account, the network would look like this:

      ------- edge_1---- vertex_0 ------ edge_5 -----
      |                    |                       |
      |                  edge_3                    |
      |                    |                       |
    vertex_2            vertex_4                vertex_6
      |
    edge_9
      |
    vertex_10  

To use the graph_processing package for this network, you can extract the corresponding data and input it to the function as follow:

In [8]:
edge_ids = [1, 3, 5, 7, 8, 9]
edge_vertex_id = [(0, 2), (0, 4), (0, 6), (2, 4), (4, 6), (2, 10)]
edge_enabled = [True, True, True, False, False, True]
source_id = 0

result = GraphProcessor(
    edge_ids=edge_ids,
    edge_vertex_id_pairs=edge_vertex_id,
    edge_enabled=edge_enabled,
    source_vertex_id=source_id,
)